In [1]:
import pandas as pd
import numpy as np

In [2]:
db = pd.read_csv('train_music.csv')
db.head()

,userId,id,artistId
0,-9223365420690745920,4789011704403477758,4862000404086590321
1,-9223357012432074795,-3685301030413601284,-5911062739952260612
2,-9223357012432074795,-1040959798245656474,6738173673409941080
3,-9223331337535253792,-8750332497321603794,-2531033142411854446
4,-9223331337535253792,7353026449734842135,-285672225852509948


In [3]:
db = db.astype(object)

In [7]:
from catboost import CatBoostRanker, Pool

In [13]:
y_train = [1,]*len(db)

In [23]:
model = CatBoostRanker(iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='PairLogit',
    eval_metric='MRR')

In [31]:
train = Pool(db['userId'], label=db['id'], group_id=db['artistId'].query())

AttributeError: 'Series' object has no attribute 'query'

In [11]:
y_train = Pool(y_train)

In [26]:
model.fit(train)

CatBoostError: catboost/private/libs/pairs/util.cpp:94: Cannot generate pairs for data without groups

In [33]:
db = db.iloc[1:]
db.head()

,user_id,item_id,Artist
1,-9223365420690745920,4789011704403477758,4862000404086590321
2,-9223357012432074795,-3685301030413601284,-5911062739952260612
3,-9223357012432074795,-1040959798245656474,6738173673409941080
4,-9223331337535253792,-8750332497321603794,-2531033142411854446
5,-9223331337535253792,7353026449734842135,-285672225852509948


In [58]:
db[Columns.Datetime] = range(len(db))
db.head()

,user_id,item_id,Artist,datetime,weight
1,-9223365420690745920,4789011704403477758,4862000404086590321,0,1
2,-9223357012432074795,-3685301030413601284,-5911062739952260612,1,1
3,-9223357012432074795,-1040959798245656474,6738173673409941080,2,1
4,-9223331337535253792,-8750332497321603794,-2531033142411854446,3,1
5,-9223331337535253792,7353026449734842135,-285672225852509948,4,1


In [36]:
db[Columns.Weight] = 1

In [59]:
ds = Dataset.construct(db,cat_item_features=['Artist'], make_dense_item_features=False)

In [38]:
from rectools.models.lightfm import LightFMWrapperModel
from rectools.models.lightfm import LightFM

In [61]:
light_fm = LightFM()
model = LightFMWrapperModel(light_fm, epochs=300, num_threads=16, verbose=1)

In [62]:
model.fit(ds)

Epoch: 100%|██████████████████████████████████████████████████████████████████████████| 300/300 [03:38<00:00,  1.37it/s]


In [63]:
submission = pd.read_csv('sub_music.csv',
                        header=None,
                        names=[Columns.User, 'recommended_id_1', 'recommended_id_2', 'recommended_id_3']).iloc[1:]
submission.head()

/tmp/ipykernel_787/3009147569.py:1: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  submission = pd.read_csv('sub_music.csv',


,user_id,recommended_id_1,recommended_id_2,recommended_id_3
1,-9223331337535253792,-2227263297444998835,-4695036104375440069,-8860367773155756475
2,-9223315435871723421,-2227263297444998835,-4695036104375440069,-8860367773155756475
3,-9223312953116183861,-2227263297444998835,-4695036104375440069,-8860367773155756475
4,-9223189477397287224,-2227263297444998835,-4695036104375440069,-8860367773155756475
5,-9223099495950652905,-2227263297444998835,-4695036104375440069,-8860367773155756475


In [64]:
recs = model.recommend(users=submission[Columns.User].unique(),
                      dataset=ds,
                      k=3,
                      filter_viewed=True)
recs

,user_id,item_id,score,rank
0,-9223331337535253792,975260989596374069,10.318725,1
1,-9223331337535253792,-2227263297444998835,10.292910,2
2,-9223331337535253792,-8860367773155756475,10.281348,3
3,-9223312953116183861,975260989596374069,11.900080,1
4,-9223312953116183861,-2227263297444998835,11.885365,2
...,...,...,...,...
1166683,9222609400741559907,-8860367773155756475,9.502502,2
1166684,9222609400741559907,975260989596374069,9.499928,3
1166685,9223256937547721480,-2227263297444998835,9.505263,1
1166686,9223256937547721480,-8860367773155756475,9.502502,2


In [65]:
recs.head(12)

,user_id,item_id,score,rank
0,-9223331337535253792,975260989596374069,10.318725,1
1,-9223331337535253792,-2227263297444998835,10.292910,2
2,-9223331337535253792,-8860367773155756475,10.281348,3
3,-9223312953116183861,975260989596374069,11.900080,1
4,-9223312953116183861,-2227263297444998835,11.885365,2
5,-9223312953116183861,-2471246448647356451,11.848369,3
6,-9223099495950652905,-8860367773155756475,10.833808,1
7,-9223099495950652905,975260989596374069,10.833379,2
8,-9223099495950652905,-2227263297444998835,10.799128,3
9,-9222957206048518287,-8860367773155756475,10.866559,1


In [66]:
preds = []
k = 0
lst = []
for x in recs[Columns.Item]:
    if len(lst) == 3:
        preds.append(lst)
        lst = []
    lst.append(int(x))
preds[0:4]

[[975260989596374069, -2227263297444998835, -8860367773155756475],
 [975260989596374069, -2227263297444998835, -2471246448647356451],
 [-8860367773155756475, 975260989596374069, -2227263297444998835],
 [-8860367773155756475, 975260989596374069, -2227263297444998835]]

In [67]:
preds[12333]

[-2227263297444998835, 975260989596374069, -8860367773155756475]

In [68]:
db['item_id'].value_counts()

item_id
-2227263297444998835    11092
-8860367773155756475    10989
-4695036104375440069    10672
 975260989596374069     10050
-2471246448647356451     9335
                        ...  
-5528982133512414188        1
 1947470080445177495        1
-6601807804298322716        1
-4184003162396588276        1
 2123071927435539666        1
Name: count, Length: 712748, dtype: int64

In [69]:
for i, x in enumerate(preds):
    c1, c2, c3 = x
    submission.iloc[i, 1] = c1
    submission.iloc[i, 2] = c2
    submission.iloc[i, 3] = c3

In [70]:
submission.head()

,user_id,recommended_id_1,recommended_id_2,recommended_id_3
1,-9223331337535253792,975260989596374069,-2227263297444998835,-8860367773155756475
2,-9223315435871723421,975260989596374069,-2227263297444998835,-2471246448647356451
3,-9223312953116183861,-8860367773155756475,975260989596374069,-2227263297444998835
4,-9223189477397287224,-8860367773155756475,975260989596374069,-2227263297444998835
5,-9223099495950652905,-8860367773155756475,-2227263297444998835,975260989596374069


In [72]:
submission['recommended_id_1'].value_counts()

recommended_id_1
-2227263297444998835    265389
-8860367773155756475     71496
975260989596374069       50731
-4695036104375440069      1136
-980265047486972863        127
-2471246448647356451        15
-7222806403183889433         1
-3606742973010966116         1
Name: count, dtype: int64

In [73]:
submission = submission.rename(columns={'user_id': 'userId'})

In [74]:
submission.to_csv('res_music2.csv',
                 index=False)